In [1]:
# !pip install web3
import dotenv
import os
from web3 import Web3
import requests
import pandas as pd
from tqdm import tqdm

os.environ['ETHERSCAN_API_KEY']='<give you api key here>'
dotenv.load_dotenv(".env", override=True)

False

# scrap hacked address 

In [2]:
import requests
from bs4 import BeautifulSoup
import time

# Define headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Function to scrape Ethereum addresses from a specific page
def scrape_addresses(page_number):
    url = f'https://etherscan.io/accounts/{page_number}?l=Phish%2fHack&ps=100'
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check if request was successful

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table containing the addresses
    table = soup.find('table', {'class': 'table'})
    if not table:
        print(f"Table not found on page {page_number}")
        return []
    
    # Extract addresses from the table rows
    addresses = []
    rows = table.find_all('tr')
    for row in rows[1:]:  # Skip header row
        cols = row.find_all('td')
        if len(cols) > 1:
            address = cols[1].text.strip()
            addresses.append(address)
    
    return addresses

# Loop through pages and collect addresses
all_addresses = []
for page_number in range(1, 3):  # Adjust the range for more pages
    try:
        page_addresses = scrape_addresses(page_number)
        all_addresses.extend(page_addresses)
        print(f"Scraped {len(page_addresses)} addresses from page {page_number}")
        time.sleep(2)  # Sleep to avoid hitting rate limits
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error on page {page_number}: {e}")
        break

# Output collected addresses
print("All Scraped Addresses:")
for address in all_addresses:
    print(address)


Scraped 100 addresses from page 1
Scraped 100 addresses from page 2
All Scraped Addresses:
0x00000000...03d7705Fa
0xC02aaA39...83C756Cc2
0xBE0eB53F...2404d33E8
0x40B38765...6e418E489
0x8315177a...4DBd7ed3a
0x49048044...fAF74E97e
0xDA9dfA13...6f6EA73Cf
0x47ac0Fb4...507a6D503
0xF977814e...97441aceC
0xa6715EAF...0A7C90033
0xbEb5Fc57...e41f106Ed
0x742d35Cc...e4438f44e
0xC61b9BB3...aeDCE193C
0xE92d1A43...88f85226f
0x61EDCDf5...1f1a56eEA
0x1Db92e2E...BcD2dFCF4
0xcA8Fa8f0...97c8aFfCa
0x0E58e899...893d9BfCD
0x3BfC20f0...F16540258
0x81036832...d865C225E
0xde0B2956...4cb697BAe
0x8696e84a...a9648C8C2
0x0a4c79cE...d83C44c76
0x2B6eD29A...51080Ffb9
0x539C9218...95baBBcA1
0x868daB0b...6c78C6d7F
0xbFbBFacC...80f3BD10F
0xD3a22590...f0404C4A1
0x1b3cB81E...ac763A7C2
0xAFCD96e5...ACD45C5dA
0x220866B1...268cA3A9D
0x0C23fc0E...045581F2d
0x94dBF04E...f86560C74
0xbF3aEB96...e7c3Fdd28
0xf3B0073E...222C302A3
0x5B5B69f4...897bc7eC4
0x2F2D854c...bB42c9b10
0x15C22df3...d0F8B38A1
0x999E77c9...813A9946E
0x8484Ef72..

# recommend manually downlaod from the page or find another way since if do that its shortened like 0xBf94F0AC...b2cbb420B
# or you can download from gcp mainnent transaction data

In [4]:
address_list = pd.read_csv('./export-accounts-1730613121530.csv')
list_of_address = address_list['Address'].tolist() 
len(list_of_address)

100

# from gcp mainnent transaction data (assume there is no fraud)

In [5]:
address_list_flag0 = pd.read_csv('./bquxjob_2927f56b_192f07f11e5.csv')
list_of_address_0 = address_list_flag0['from_address'].tolist() 
len(list_of_address_0)

100

In [36]:
# address='0x87d884aaa6ff9e9b6014631b0abae80b53953fb8'
# msg=requests.get(formQueryString(address,1,0,os.environ.get("ETHERSCAN_API_KEY")))
# msg.content

In [7]:

def formQueryString(address, pgNo, offset, api_key):
    return "https://api.etherscan.io/api?module=account&action=txlist&address=" + address + "&startblock=0&endblock=99999999&page=" + str(pgNo) + "&offset=" + str(offset) + "&sort=asc&apikey=" + os.environ.get("ETHERSCAN_API_KEY")

def get_address_stats_normal_tnx(address, hacked):
    response = requests.get(formQueryString(address,1,0,os.environ.get("ETHERSCAN_API_KEY")))

    sample_df = pd.DataFrame(response.json()['result'])
    # Column creation of ETH  
    sample_df['eth value'] = sample_df['value'].apply(lambda x: Web3.from_wei(int(x),'ether'))

    # Typing of sent and received transactions
    sample_df['txn type'] = sample_df['from'].apply(lambda x: 'sent' if x== address else 'received')

    # Handling of Sent transactions stats
    sample_df_sent =  sample_df[sample_df['txn type'] == 'sent']
    sample_df_sent = sample_df_sent.sort_values(by=['timeStamp'])
    sample_df_sent['timeStamp'] = sample_df_sent['timeStamp'].astype('int')

    # Filtering of sent normal transfers to contract addresses
    sample_df_sent_contracts = sample_df[sample_df['contractAddress'] != '']

    # Compilation of normal sent transaction statistics
    core_stats_Sent_tnx = len(sample_df_sent)
    core_stats_MinValSent = sample_df_sent['eth value'].min()
    core_stats_MaxValSent = sample_df_sent['eth value'].max()
    core_stats_AvgValSent = sample_df_sent['eth value'].mean()
    core_stats_MinValueSentToContract = sample_df_sent_contracts['eth value'].min()
    core_stats_MaxValueSentToContract = sample_df_sent_contracts['eth value'].max()
    core_stats_AvgValueSentToContract = sample_df_sent_contracts['eth value'].mean()
    core_stats_TotalEtherSent = sample_df_sent['eth value'].sum()
    core_stats_TotalEtherSent_Contracts = sample_df_sent_contracts['eth value'].sum()
    core_stats_UniqueSentTo_Addresses = len(sample_df_sent['to'].unique())

    # Handling of received transactions stats
    sample_df_received =  sample_df[sample_df['txn type'] == 'received']
    sample_df_received = sample_df_received.sort_values(by=['timeStamp'])
    sample_df_received['timeStamp'] = sample_df_received['timeStamp'].astype('int')


    # Compilation of normal received transaction statistics
    core_stats_Received_tnx = len(sample_df_received)
    core_stats_MinValueReceived = sample_df_received['eth value'].min()
    core_stats_MaxValueReceived = sample_df_received['eth value'].max()
    core_stats_AvgValueReceived = sample_df_received['eth value'].mean()
    core_stats_TotalEtherReceived = sample_df_received['eth value'].sum()
    core_stats_UniqueReceivedFrom_Addresses = len(sample_df_received['from'].unique())

    # Handling of remaining normal transaction values
    sample_df['timeStamp'] = sample_df['timeStamp'].astype('int')
    sample_df.sort_values(by=['timeStamp'])
    sample_df['unix time difference'] = sample_df['timeStamp'].diff()
    sample_df_time_dim = sample_df.groupby('txn type')['unix time difference'].sum()/60


    # Compilation of remaining normal transaction statistics
    core_stats_TimeDiffbetweenfirstand_last = ((sample_df['timeStamp'].max()) - (sample_df['timeStamp'].min())) / 60
    core_stats_TotalTransactions = len(sample_df)
    core_stats_NumberofCreated_Contracts = len(sample_df[sample_df['contractAddress'] != ''])
    core_stats_Avg_min_between_received_tnx = sample_df_time_dim['received']/ core_stats_Received_tnx
    core_stats_Avg_min_between_sent_tnx = sample_df_time_dim['sent']/core_stats_Sent_tnx
    core_stats_TotalEtherBalance = core_stats_TotalEtherReceived - core_stats_TotalEtherSent
    compiled_normal_tnx_result = {'Address': address, 'FLAG': hacked,
                                  'Avg min between sent tnx': core_stats_Avg_min_between_sent_tnx,
                                 'Avg min between received tnx': core_stats_Avg_min_between_received_tnx,
                                  'Time Diff between first and last (Mins)': core_stats_TimeDiffbetweenfirstand_last,
                                  'Sent tnx': core_stats_Sent_tnx, 'Received Tnx': core_stats_Received_tnx,
                                  'Number of Created Contracts': core_stats_NumberofCreated_Contracts,
                                  'Unique Received From Addresses':core_stats_UniqueReceivedFrom_Addresses,
                                  'Unique Sent To Addresses': core_stats_UniqueSentTo_Addresses,
                                  'min value received': core_stats_MinValueReceived,
                                  'max value received ': core_stats_MaxValueReceived,
                                  'avg val received': core_stats_AvgValueReceived,
                                  'min val sent': core_stats_MinValSent,
                                  'max val sent': core_stats_MaxValSent,
                                  'avg val sent': core_stats_AvgValSent ,
                                  'min value sent to contract': core_stats_MinValueSentToContract,
                                  'max val sent to contract': core_stats_MaxValueSentToContract,
                                  'avg value sent to contract': core_stats_AvgValueSentToContract,
                                  'total transactions (including tnx to create contract': core_stats_TotalTransactions,
                                  'total Ether sent': core_stats_TotalEtherSent,
                                  'total ether received': core_stats_TotalEtherReceived,
                                  'total ether sent contracts': core_stats_TotalEtherSent_Contracts,
                                  'total ether balance':core_stats_TotalEtherBalance}
    return pd.DataFrame([compiled_normal_tnx_result])

def get_empty_details_for_address(address, hacked):
    compiled_empty_address = {
          'Address': address, 'FLAG': hacked,
          'Avg min between sent tnx': 0,
         'Avg min between received tnx': 0,
          'Time Diff between first and last (Mins)': 0,
          'Sent tnx': 0, 'Received Tnx': 0,
          'Number of Created Contracts': 0,
          'Unique Received From Addresses':0,
          'Unique Sent To Addresses': 0,
          'min value received': 0,
          'max value received ': 0,
          'avg val received': 0,
          'min val sent': 0,
          'max val sent': 0,
          'avg val sent': 0 ,
          'min value sent to contract': 0,
          'max val sent to contract': 0,
          'avg value sent to contract': 0,
          'total transactions (including tnx to create contract': 0,
          'total Ether sent': 0,
          'total ether received': 0,
          'total ether sent contracts': 0,
          'total ether balance':0
    }
    return pd.DataFrame([compiled_empty_address])



# hacked

In [8]:
write_to_file = './address_data.csv'

base_df = pd.DataFrame()
total_transactions = 0
for i in tqdm(range(len(list_of_address))): #
    a = list_of_address[i]
    try:
        if i==0:
            base_df = get_address_stats_normal_tnx(a,hacked=1)
            base_df.to_csv(write_to_file,mode='w',index=False, header=True)
            itxns = base_df.loc[0,'total transactions (including tnx to create contract']
            total_transactions = itxns
            print("Address number {}: {} mined! {} retrieved. {} total transactions.".format(i,a,itxns,total_transactions))
        else:
            cand_df = get_address_stats_normal_tnx(a,hacked=1)
            cand_df.to_csv(write_to_file,mode='a', index=False,header=False)
            base_df = pd.concat([base_df, cand_df])
            itxns = cand_df.loc[0,'total transactions (including tnx to create contract']
            total_transactions = total_transactions + itxns
            print("Address number {}: {} mined! {} retrieved. {} total transactions.".format(i,a,itxns,total_transactions))

    except:
        cand_df = get_empty_details_for_address(a,hacked=1)
        base_df = pd.concat([base_df, cand_df])
        cand_df.to_csv(write_to_file,mode='a',index=False,header=False)
        print("Address number {}: {} mined! 0 txns retrieved. {} total transactions.".format(i,a,total_transactions))

base_df = base_df.reset_index(drop=True)



  1%|▍                                          | 1/100 [00:05<09:13,  5.59s/it]

Address number 0: 0x00000000219ab540356cbb839cbe05303d7705fa mined! 0 txns retrieved. 0 total transactions.


  2%|▊                                          | 2/100 [00:11<09:45,  5.98s/it]

Address number 1: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 mined! 0 txns retrieved. 0 total transactions.


  3%|█▎                                         | 3/100 [00:14<06:55,  4.28s/it]

...

Address number 97: 0xe9a5a2acfa9bee149ed28fcbf12b60ff2ad97efb mined! 0 txns retrieved. 60554 total transactions.


 99%|█████████████████████████████████████████▌| 99/100 [03:16<00:00,  1.08it/s]

Address number 98: 0xe93fc974908003faddf0bd972b05b6494368d3f5 mined! 0 txns retrieved. 60554 total transactions.


100%|█████████████████████████████████████████| 100/100 [03:17<00:00,  1.98s/it]

Address number 99: 0x303c741bb15c9c4b1c72320f6df702b32da80519 mined! 0 txns retrieved. 60554 total transactions.


# non hack 

In [ ]:
base_df_0 = pd.DataFrame()
total_transactions = 0
for i in tqdm(range(len(list_of_address))):
    a = list_of_address[i]
    try:
        if i == 0:
            base_df_0 = get_address_stats_normal_tnx(a, hacked=1)
            base_df_0.to_csv(write_to_file, mode='w', index=False, header=True)
            itxns = base_df_0.loc[0, 'total transactions (including tnx to create contract']
            total_transactions = itxns
            print("Address number {}: {} mined! {} retrieved. {} total transactions.".format(i, a, itxns, total_transactions))
        else:
            cand_df = get_address_stats_normal_tnx(a, hacked=1)
            cand_df.to_csv(write_to_file, mode='a', index=False, header=False)
            base_df_0 = pd.concat([base_df_0, cand_df])
            itxns = cand_df.loc[0, 'total transactions (including tnx to create contract']
            total_transactions += itxns
            print("Address number {}: {} mined! {} retrieved. {} total transactions.".format(i, a, itxns, total_transactions))

    except:
        cand_df = get_empty_details_for_address(a, hacked=1)
        base_df_0 = pd.concat([base_df_0, cand_df])
        cand_df.to_csv(write_to_file, mode='a', index=False, header=False)
        print("Address number {}: {} mined! 0 txns retrieved. {} total transactions.".format(i, a, total_transactions))

base_df_0 = base_df_0.reset_index(drop=True)


  1%|▍                                          | 1/100 [00:01<02:32,  1.54s/it]

Address number 0: 0xe6182a6aa6513e24232690f3db97b5535baa3ea4 mined! 199 retrieved. 199 total transactions.


  2%|▊                                          | 2/100 [00:04<04:15,  2.61s/it]

Address number 1: 0xbab2f87bcc50b8abc542ed3bf152043f74f6910d mined! 4802 retrieved. 5001 total transactions.


  3%|█▎                                         | 3/100 [00:05<03:02,  1.88s/it]

...

Address number 33: 0x8e670b4d6651c4051e65b21aa4a575f3f99b8b83 mined! 10000 retrieved. 76665 total transactions.


 35%|██████████████▋                           | 35/100 [01:48<02:46,  2.56s/it]

Address number 34: 0xd9a0c3969c56309be72881b14f569a096794eb9e mined! 761 retrieved. 77426 total transactions.


 36%|███████████████                           | 36/100 [01:50<02:31,  2.37s/it]

Address number 35: 0xe64b9fbc6ed0f8df53738d773156732956f96b67 mined! 1084 retrieved. 78510 total transactions.


In [ ]:
# Concatenate DataFrames vertically
combined_df = pd.concat([based_df, base_df_0], axis=0, ignore_index=True)

# Save to CSV
combined_df.to_csv('data.csv', index=False)